# 第十一章 特征工程之数据预处理（11.6至11.7）

# 11.6 多重共线性的分析与处理

In [1]:
# 使用Pandas库读入一组存在多重共线性的数据，并对其回归作为示例：
import pandas as pd
df = pd.read_excel('数据.xlsx')
df.head()

,X1,X2,X3,Y
0,8,16,-32,77
1,7,14,-31,52
2,4,9,-12,42
3,1,2,5,6
4,1,2,8,19


In [2]:
# 对数据集划分特征变量和目标变量：
X = df.drop(columns='Y')
Y = df['Y']

下面我们需要做的就是分析与检验这三个特征变量是否存在多重共线性，这里主要讲解两种判别方法：相关系数判断以及方差膨胀因子法（VIF检验）来检验多重共线性。

1.相关系数判断

In [3]:
X.corr()

,X1,X2,X3
X1,1.000000,0.992956,-0.422788
X2,0.992956,1.000000,-0.410412
X3,-0.422788,-0.410412,1.000000


其中第i行第j列的内容表示的就是对应的第i个特征变量和第j个特征变量的相关系数，例如第1行第2列的相关系数：0.99表示的就是特征变量X1和特征变量X2的相关系数，可以看到这两个特征变量的相关性还是非常强的，因此有理由相信这两个变量会导致多重共线性的现象，因此需要删去其中一个特征变量。注意对角线上的相关系数都为1，这个其实没有什么意义，因为它表示的是自身与自身的相关系数，那自然是1了。

2.方差膨胀因子法（VIF检验）

In [4]:
# 为了检验上述回归中是否存在严重的多重共线性，我们使用Python的VIF检验模块来验证：
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = [variance_inflation_factor(X.values, X.columns.get_loc(i)) for i in X.columns]

In [5]:
vif

[259.6430487184967, 257.6315718292196, 1.302330632715429]

In [6]:
# 如果对上面的快捷写法不太理解，上面的代码也可以写成：
vif = []
for i in X.columns:  # i对应的是每一列的列名
    vif.append(variance_inflation_factor(X.values, X.columns.get_loc(i)))
    
vif

[259.6430487184967, 257.6315718292196, 1.302330632715429]

In [7]:
# 对数据集重新划分特征变量和目标变量：
X = df[['X1', 'X3']]
Y = df['Y']

In [8]:
# 进行检验VIF检验：
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = [variance_inflation_factor(X.values, X.columns.get_loc(i)) for i in X.columns]

vif

[1.289349054516766, 1.289349054516766]

# 11.7 过采样和欠采样

11.7.1 过采样

In [9]:
import pandas as pd
data = pd.read_excel("信用卡数据.xlsx")
data.head()

,编号,年龄,负债比率,月收入,贷款数量,家属人数,分类
0,1,29,0.22,7800,1,3,0
1,2,52,0.46,4650,1,0,0
2,3,28,0.10,3000,0,0,0
3,4,29,0.20,5916,0,0,0
4,5,27,1.28,1300,0,0,1


In [10]:
# 通过如下代码提取特征变量并将其赋值给变量X，提取目标变量并将其赋值给变量y：。
X = data.drop(columns='分类')
y = data['分类']

In [11]:
# 然后使用collections库中的Counter()方法，对目标变量进行计数：
from collections import Counter
Counter(y)

Counter({0: 1000, 1: 100})

不违约的样本数有1000个，远远大于违约的样本数100。为了防止建立信用违约模型时，模型着重拟合不违约的样本，而无法找出违约的样本，我们采用过采样的方法来改善样本比例不均衡的问题，这里我们将通过上面讲到的随机过采样和SMOTE法过采样来进行代码实现。

（1）随机过采样

In [12]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_oversampled, y_oversampled = ros.fit_resample(X, y)

In [13]:
# 用Counter()方法检验一下SMOTE过采样的效果。
Counter(y_oversampled)

Counter({0: 1000, 1: 1000})

违约的样本数从100上升至不违约的样本数1000，这证明我们的随机过采样有效。同时我们可以打印特征变量X_oversampled的shape来看看特征变量的变化：

In [14]:
X_oversampled.shape

(2000, 6)

这里的2000就是1000个违约样本和1000个不违约样本相加得到的，可以看到，随机过采样后特征变量的数据也随之增多。

（2）SMOTE过采样

In [15]:
# 通过如下代码即可进行SMOTE过采样：
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)
X_smotesampled, y_smotesampled = smote.fit_resample(X, y)

In [16]:
# 用Counter()方法检验一下SMOTE过采样的效果。
Counter(y_smotesampled)

Counter({0: 1000, 1: 1000})

11.7.2 欠采样

In [17]:
# 仍然采用上方的信用违约数据进行欠采样代码的展示：
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_undersampled, y_undersampled = rus.fit_resample(X, y)

In [18]:
# 用Counter()方法检验一下随机欠采样的效果。
Counter(y_undersampled)

Counter({0: 100, 1: 100})

In [19]:
# 不违约的样本数从1000下降至违约的样本数100，这证明我们的随机欠采样有效。同时我们可以打印特征变量X_undersampled的shape来看看特征变量的变化：
X_undersampled.shape

(200, 6)